# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

df = pd.read_csv("homepage_actions.csv")

print(df.head())
print(df.info())
print(df['id'].nunique())

viewers = df[df['action'] == 'view']['id'].nunique()
clickers = df[df['action'] == 'click']['id'].nunique()
print("Viewers:", viewers, " Clickers:", clickers)

click_without_view = set(df[df['action'] == 'click']['id']) - set(df[df['action'] == 'view']['id'])
print("Click without view:", len(click_without_view))

overlap = set(df[df['group'] == 'control']['id']).intersection(set(df[df['group'] == 'experiment']['id']))
print("Overlap:", len(overlap))

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
6328
Viewers: 6328  Clickers: 1860
Click without view: 0
Overlap: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
import scipy.stats as stats
from math import sqrt

user_df = df.groupby(['id','group','action']).size().unstack().reset_index().fillna(0)
user_df['clicked'] = (user_df['click'] > 0).astype(int)

control = user_df[user_df['group'] == 'control']
experiment = user_df[user_df['group'] == 'experiment']

p_control = control['clicked'].mean()
p_experiment = experiment['clicked'].mean()

n_control = control.shape[0]
n_experiment = experiment.shape[0]

p_pool = (control['clicked'].sum() + experiment['clicked'].sum()) / (n_control + n_experiment)
se = sqrt(p_pool * (1 - p_pool) * (1/n_control + 1/n_experiment))

z = (p_experiment - p_control) / se
p_value = 1 - stats.norm.cdf(z)

print("z:", z, " p-value:", p_value)

z: 2.618563885349469  p-value: 0.004415037788297882


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
#Your code here
expected_clicks_exp = n_experiment * p_control
print(expected_clicks_exp)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
#Your code here
actual_clicks_exp = experiment['clicked'].sum()
std = sqrt(n_experiment * p_control * (1 - p_control))
z_manual = (actual_clicks_exp - expected_clicks_exp) / std
print(z_manual)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
#Your code here
p_value_manual = 1 - stats.norm.cdf(z_manual)
print(p_value_manual)

0.00012486528006949715


In [8]:
### Analysis:

#Does this result roughly match that of the previous statistical test?

# Comment: **Your analysis here**
# Analysis:
# The p-value is small enough to reject the null hypothesis, meaning
# the experimental homepage performed significantly better than the control.
# Both the initial z-test and the manual verification give very similar results,
# confirming the reliability of the conclusion.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.